<a href="https://colab.research.google.com/github/450586509/practical-ml/blob/master/tensorflow_Mask%E6%9C%BA%E5%88%B6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import tensorflow as tf
import tensorflow.keras.layers as L

sess = tf.InteractiveSession()

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
raw_inputs = [
  [83, 91, 1, 645, 1253, 927],
  [73, 8, 3215, 55, 927],
  [71, 1331, 4231]
]

In [42]:
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,padding='post')
print(padded_inputs)

[[  83   91    1  645 1253  927]
 [  73    8 3215   55  927    0]
 [  71 1331 4231    0    0    0]]


### 有三种Mask机制


*   利用keras.layers.Masking
*   配置keras.layers.Embedding的mask_zero=True
*   对支持Mask的层，传递mask参数



#### 验证Embedding层

In [43]:
embedding = L.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

Tensor("embedding_3/NotEqual:0", shape=(3, 6), dtype=bool)


In [0]:
init = tf.global_variables_initializer()
sess.run(init)

In [45]:
## 为什么在Embedding中设置mask_zero=True,会给后面的层带来mask效果，因为
## 输出的tensor中有_keras_mask成员，表示mask信息。
sess.run(masked_output._keras_mask)

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True, False],
       [ True,  True,  True, False, False, False]])

#### 验证Masking层


原理：被masking的tensor增加了一个属性：._keras_mask，表示哪些被mask掉

In [65]:
masking_layer = L.Masking()
# Simulate the embedding lookup by expanding the 2D input to 3D,
# with embedding dimension of 10.
unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]),
    tf.float32)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

Tensor("masking_2/Squeeze:0", shape=(3, 6), dtype=bool)


In [66]:
print("masked_embedding._keras_mask={}".format(masked_embedding._keras_mask.eval()))

masked_embedding._keras_mask=[[ True  True  True  True  True  True]
 [ True  True  True  True  True False]
 [ True  True  True False False False]]


In [0]:
###

### 第二个lstm是否有mask信息

结论：mask信息在mask层中可以传递

In [0]:
lstm_first = L.LSTM(units=10,return_sequences=True,activation='sigmoid')
lstm_second = L.LSTM(units=1, return_sequences=True, activation='sigmoid')

In [0]:

lstm_first_out = lstm_first(masked_embedding)
lstm_second_out = lstm_second(lstm_first_out)

In [86]:
init = tf.global_variables_initializer()
sess.run(init)
print("lstm_first_out._keras_mask={}".format(lstm_first_out._keras_mask.eval()))
print("lstm_second_out._keras_mask={}".format(lstm_second_out._keras_mask.eval()))

lstm_first_out._keras_mask=[[ True  True  True  True  True  True]
 [ True  True  True  True  True False]
 [ True  True  True False False False]]
lstm_second_out._keras_mask=[[ True  True  True  True  True  True]
 [ True  True  True  True  True False]
 [ True  True  True False False False]]


In [83]:
lstm_second_out._keras_mask.eval()

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True, False],
       [ True,  True,  True, False, False, False]])